In [1]:
from pyspark import SparkConf, SparkContext

class WikipediaArticle:
    def __init__(self, title: str, text: str):
        self.title = title
        self.text = text

def parse(line: str):
    subs = "</title><text>"
    i = line.find(subs)
    title = line[14:i]
    text = line[i + len(subs):-16]
    return WikipediaArticle(title, text)

In [2]:
conf = SparkConf().setAppName("Wikipedia Languages")
sc = SparkContext(conf=conf)

languages = ["JavaScript", "Java", "PHP", "Python", "C#", "C++", "Ruby", "CSS", "Objective-C", "Perl", "Scala", "Haskell", "MATLAB", "Clojure", "Groovy"]

23/01/17 22:13:22 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
data = sc.textFile("wikipedia.dat")

In [4]:
# count how many times each of the programming languages occurs

def count_languages(article: WikipediaArticle):
    count = {}
    for language in languages:
        count[language] = article.text.count(language)
    return count.items()

language_counts = data.map(parse).flatMap(count_languages)

language_counts = language_counts.reduceByKey(lambda a, b: a + b)

results = language_counts.collect()
for result in results:
    print(f"{result[0]}: {result[1]}")

C++: 3044
Java: 15306
Ruby: 1233
PHP: 1918
Haskell: 561
C#: 2781
CSS: 3463
Objective-C: 640
Perl: 1060
Scala: 523
JavaScript: 7488
Python: 2523
MATLAB: 1004
Clojure: 123
Groovy: 178


In [5]:
# for each language, in how many articles is it mentioned

def count_articles_by_language(article: WikipediaArticle):
    count = {}
    for language in languages:
        if language in article.text:
            count[language] = 1
    return count.items()

articles_counts = data.map(parse).flatMap(count_articles_by_language)

articles_counts = articles_counts.reduceByKey(lambda a, b: a + b)

results = articles_counts.collect()
for result in results:
    print(f"{result[0]}: {result[1]}")

C++: 554
Ruby: 287
Java: 2017
PHP: 452
Haskell: 128
CSS: 555
Perl: 300
Scala: 161
C#: 849
Objective-C: 112
JavaScript: 1738
MATLAB: 324
Python: 545
Groovy: 55
Clojure: 60
